In [ ]:
%use coroutines

import kotlinx.coroutines.flow.*
import pl.mareklangiewicz.kommand.*
import pl.mareklangiewicz.kommand.CliPlatform.Companion.SYS
import pl.mareklangiewicz.kommand.core.*
import pl.mareklangiewicz.kommand.core.LsOpt.*
import pl.mareklangiewicz.kommand.gnome.*
import pl.mareklangiewicz.kommand.gnome.GnomeExt.Cmd.list
import kotlin.time.*

@OptIn(ExperimentalTime::class)
fun Flow<*>.logEachWithMillisBlocking() = runBlocking { logEachWithMillis() }

// TODO NOW: go through this whole notebook again (commented versions too) after refactor; also test input/output sequences (like slow find output)
// TODO NOW: analyze ProcessBuilder.startPipeline and understand assumptions/invariants and make sure my "pipelines" with line sequences are correct.
//    also reproduce example from ProcessBuilder.startPipeline javadoc using my stuff

In [ ]:
    /**
     * Kinda like .exec, but less strict/explicit, because here in notebook we are in more local "experimental" context.
     * 
     * WARNING: Current impl first wait for process to read whole input (blocking) and then starts to consume output.
     * If it deadlocks, that is why.. See CliPlatform.execonsume - same problem
     */
@OptIn(DelicateKommandApi::class, DelicateCoroutinesApi::class)
fun Kommand.x(
    platform: CliPlatform = SYS,
    dir: String? = null,
    vararg useNamedArgs: Unit,
    inContent: String? = null,
    inLinesFlow: Flow<String>? = inContent?.lineSequence()?.asFlow(),
    inFile: String? = null,
    outFile: String? = null,
    outFileAppend: Boolean = false,
    errToOut: Boolean = false,
    errFile: String? = null,
    errFileAppend: Boolean = false,
    expectedExit: Int? = 0,
    expectedErr: ((List<String>) -> Boolean)? = { it.isEmpty() },
    outLinesCollector: FlowCollector<String>? = null,
): List<String> = runBlocking {
    require(platform.isRedirectFileSupported || (inFile == null && outFile == null)) { "redirect file not supported here" }
    require(inLinesFlow == null || inFile == null) { "Either inLinesFlow or inFile or none, but not both" }
    require(outLinesCollector == null || outFile == null) { "Either outLinesCollector or outFile or none, but not both" }
    val eprocess = platform.start(this@x, 
        dir = dir, 
        inFile = inFile, 
        outFile = outFile, 
        outFileAppend = outFileAppend, 
        errToOut = errToOut, 
        errFile = errFile, 
        errFileAppend = errFileAppend
    )
    val inJob = inLinesFlow?.onEach(eprocess.stdin::emit)?.launchIn(this)
    val outJob = outLinesCollector?.let { eprocess.stdout.onEach(it::emit).launchIn(this) }
    inJob?.join()
    outJob?.join()
    eprocess
        .awaitResult() // inLinesFlow already used
        .unwrap(expectedExit, expectedErr)
}

In [ ]:
SYS.run {
    start(gnometerm(bash(gnomeext(list), pause = true)))
    start(gnometerm(bash(ls { -LongFormat; -All }, pause = true)))
}

In [ ]:
ls { -LongFormat; -All }.x {
    println("out line: $it")
}

In [ ]:
val k = ls { -LongFormat; -All }
val  ep = SYS.start(k)
runBlocking { 
    ep.stdout.take(3).onEach { delay(1000) }.collect { println("out line: $it") }
    ep.awaitExit()
}

In [ ]:
LsSamples.lsParentWithSlashes.x().logEach()

In [ ]:
import pl.mareklangiewicz.kommand.konfig.*

SYS.konfigInUserHomeConfigDir().loglns()

In [ ]:
import pl.mareklangiewicz.kommand.admin.*
import pl.mareklangiewicz.kommand.debian.*
import pl.mareklangiewicz.kommand.debian.DpkgAct.*
import pl.mareklangiewicz.kommand.samples.Samples

SYS.run {
    val k =
//        Samples.GitHub.help
        Samples.GitHub.secretListForAbcdK
//        Samples.GitHub.secretSetFakeSecretInAbcdK
//        dpkg { -PrintArch }
//        dpkg(Status("apt"))
//        dpkg(Details("findutils"))
//        sudo(bash("whoami; pwd; ls"))
//        sudo(kommand("aptitude"))
//        sudoEdit("/etc/hosts")
//        sudoEdit("/etc/fstab")
//        kommand("whoami").withSudo(SudoOpt.User(zenityAskForEntryExec("Enter some user name")))
    startInGnomeTermIfUserConfirms(k)
}


In [ ]:
import pl.mareklangiewicz.kommand.github.*

runBlocking { 
    SYS.run { 
        ghSecretSetTempByHand(secretName = "FAKE_SECRET_67", secretValue = "concretevalue67", repoPath = "langara/AbcdK")
     }
 }

In [ ]:
import pl.mareklangiewicz.kommand.samples.*

SYS.run {
    val rk = Samples.GitHub.secretSetConcreteFakeSecret67InAbcdK.reducedKommand
    runBlocking { 
        exec(rk)
    }
}

In [ ]:
import pl.mareklangiewicz.kommand.debian.*

SYS.dpkgSearchOneCommandExec("find")

In [ ]:
import pl.mareklangiewicz.kommand.find.*

val k = FindSamples.findInKotlinDirNodeModulesDirs
k.x {
    println(it)
}

In [ ]:
@file:Suppress("DEPRECATION")

import pl.mareklangiewicz.kommand.admin.*
import pl.mareklangiewicz.kommand.find.*
import kotlin.time.*
import kotlin.time.TimeSource.*


//    sudoExec(ls("/root/", withHidden = true), inPass = zenityAskForPasswordExec())
val k = FindSamples.run {
//    findAbcIgnoreCase
//    findSomeSamples
    findInKotlinDirNodeModulesDirs
//    findInKotlinDirBuildDirs
//    findSymLinksToKtsFilesInDepsKt
//    findDepthMax2FilesInDepsKtAndRunFileOnEach
//    findInKotlinKtFilesModifiedIn24h
//    findMyLastWeekKotlinCode
//    findInKotlinDirNodeModulesDirs
}
val p = SYS.start(k)
p.stdout.logEachWithMillisBlocking()

In [ ]:
import pl.mareklangiewicz.kommand.find.*

//val tk = findTypicalDetailsTable("..")
val tk = FindSamples.findTypicalDetailsTableInParentDir.typedKommand
val tp = SYS.start(tk)
tp.stdout.onEach { delay(200) }.take(20).logEachWithMillisBlocking()


In [ ]:
findMyKotlinCode(
    withGrepRE = "import",
    withModifTime24h = FindExpr.NumArg.LessThan(8)
).checkInIdeap()

In [ ]:
SYS.run {
    startInGnomeTermIfUserConfirms(adb(Adb.Command.pair("192.168.0.125", 38451)))
}